In [2]:
pip install simpy

Note: you may need to restart the kernel to use updated packages.


In [8]:
import simpy
import numpy as np

class AirportSecuritySystem:
    def __init__(self, env, num_id_checkers, num_personal_queues):
        self.env = env
        self.id_checkers = [simpy.Resource(env) for _ in range(num_id_checkers)]
        self.personal_queues = [simpy.Resource(env) for _ in range(num_personal_queues)]
        self.interarrival_rate = 0.2  # arrivals per minute
        self.service_rate_id_checker = 0.75  # minutes per passenger
        self.service_time_personal = lambda: np.random.uniform(0.5, 1)  # minutes
        self.total_wait_times_id = []
        self.total_wait_times_personal = []

    def passenger_arrival(self):
        while True:
            yield self.env.timeout(np.random.exponential(1 / self.interarrival_rate))
            self.env.process(self.passenger_processing())

    def passenger_processing(self):
        arrival_time = self.env.now
        with self.id_checkers[np.argmin([len(id_checker.queue) for id_checker in self.id_checkers])].request() as req:
            yield req
            id_check_wait_time = self.env.now - arrival_time
            self.total_wait_times_id.append(id_check_wait_time)
            yield self.env.timeout(np.random.exponential(1 / self.service_rate_id_checker))
            shortest_queue = np.argmin([len(personal_queue.queue) for personal_queue in self.personal_queues])
            with self.personal_queues[shortest_queue].request() as req:
                yield req
                yield self.env.timeout(self.service_time_personal())
                personal_check_wait_time = self.env.now - arrival_time
                self.total_wait_times_personal.append(personal_check_wait_time)

def simulate_airport(num_id_checkers, num_personal_queues):
    env = simpy.Environment()
    airport_system = AirportSecuritySystem(env, num_id_checkers, num_personal_queues)
    env.process(airport_system.passenger_arrival())
    env.run(until=1000)  # Run simulation for 1000 minutes

    avg_id_checker_wait_time = np.mean(airport_system.total_wait_times_id)
    avg_personal_queue_wait_time = np.mean(airport_system.total_wait_times_personal)

    return avg_id_checker_wait_time, avg_personal_queue_wait_time

# Vary the number of ID/boarding-pass checkers and personal-check queues
for num_id_checkers in range(1, 11):
    for num_personal_queues in range(1, 11):
        avg_id_checker_wait_time, avg_personal_queue_wait_time = simulate_airport(num_id_checkers, num_personal_queues)
        total_avg_wait_time = avg_id_checker_wait_time + avg_personal_queue_wait_time
        if total_avg_wait_time <= 15:
            print(f"Number of ID/boarding-pass checkers: {num_id_checkers}, Number of personal-check queues: {num_personal_queues}, Average wait time: {total_avg_wait_time} minutes")


Number of ID/boarding-pass checkers: 1, Number of personal-check queues: 1, Average wait time: 4.586249862374486 minutes
Number of ID/boarding-pass checkers: 1, Number of personal-check queues: 2, Average wait time: 4.686352889536106 minutes
Number of ID/boarding-pass checkers: 1, Number of personal-check queues: 3, Average wait time: 4.139697261683261 minutes
Number of ID/boarding-pass checkers: 1, Number of personal-check queues: 4, Average wait time: 5.12069085249614 minutes
Number of ID/boarding-pass checkers: 1, Number of personal-check queues: 5, Average wait time: 3.906807628758439 minutes
Number of ID/boarding-pass checkers: 1, Number of personal-check queues: 6, Average wait time: 4.091519113952281 minutes
Number of ID/boarding-pass checkers: 1, Number of personal-check queues: 7, Average wait time: 3.8858057842628106 minutes
Number of ID/boarding-pass checkers: 1, Number of personal-check queues: 8, Average wait time: 3.4488470221472527 minutes
Number of ID/boarding-pass chec

#### These statistics provide insights into the average wait times based on the number of ID/boarding-pass checkers and personal-check queues. Here's a summary of the trends observed:

#### As the number of personal-check queues increases:

#### With 1 ID/boarding-pass checker, the average wait time generally fluctuates but tends to hover around 4-5 minutes.
#### With 2 ID/boarding-pass checkers, the average wait time remains relatively stable, ranging from about 2.8 to 3.3 minutes.
#### With 3 or more ID/boarding-pass checkers, the average wait time tends to decrease and stabilizes around 2.8 to 3.2 minutes.
#### As the number of ID/boarding-pass checkers increases:

#### With 1 or 2 personal-check queues, the average wait time generally decreases slightly as the number of checkers increases.
#### With 3 or more personal-check queues, the average wait time remains relatively stable regardless of the number of checkers.
#### Overall, the data suggests that increasing the number of ID/boarding-pass checkers and personal-check queues can help reduce average wait times, with diminishing returns observed beyond a certain point. However, the impact of additional checkers is more pronounced when there are fewer queues, while the impact diminishes when there are more queues available.